# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
import networkx
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

Next implement a function to read in the edges from the input file.

In [2]:
def read_nodes(gfile):
    node=[]
    line=gfile.readline()
    while not (line == ''):
        if not line.startswith('*'):
            node.append(line[0:2])
        line = gfile.readline().strip()
    return node
    pass

In [3]:
def read_edges(gfile):
    edge = []
    line=gfile.readline()
    line_clean = line.replace(':',';').split(';')[1:] 
    # read a line and split for ':' and ';' and get rid of the first element 
    while not line.startswith('*'): # the last line is a comment
        for elem in line_clean:
            element = elem.strip('\n').split(',') # splitting groups
            for i in range(len(element)-1):
                edge.append((element[i],element[i+1])) 
                # for every element in a group, make edges by pairing the element with the next element
        line=gfile.readline()
        line_clean = line.replace(':',';').split(';')[1:]
    return edge
    pass

The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [5]:
def Search(graph, root):
    visited = []
    stack = [root]

    while stack:
        node = stack.pop() # node is the last element added to the stack
        if node not in visited:
            visited.append(node)
            stack += sorted(graph.neighbors(node))[::-1] 
            # index so the first alphabetical-ordered neighbor comes last we will pop it and check if it is visited 
    return visited
    pass   

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [6]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    components = []
    visited = []
    tree = []
    for i in sorted(graph.nodes()): # the node with smallest alphabetical ordering
        if i not in visited:
            tree = Search(graph, i) 
            visited += tree
            components.append(tree)
    return components
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [8]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [9]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
